# Week 15 Programming Assignment

The final thing for this semester that we haven't exercised, yet, is working with databases through Python.  In thi final assignment of the semester, you will practice pulling data from either Google Big Query or Snowflake, loading that data into a Pandas data frame, summarizing the data, and then exporting that to an Excel file.

**You will need to use your own Google Big Query or Snowflake account to run this notebook, but you should try to make it configurable so anyone with an account in those technologies could run your notebook with minimal changes.**


Build a notebook with good comments (either in code or in markdown cells).  Then submit your assignment as usual.

### 1. Connect to the data source

Follow the examples provided in the [week15 folder of our GitHub repository](https://github.com/paulboal/hds5210-2023/tree/main/week15) to connect your notebook either to Big Query or Snowflake.

In [1]:
!pip install "snowflake-connector-python[pandas]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.1.0
    Uninstalling platformdirs-4.1.0:
      Successfully uninstalled platformdirs-4.1.0


In [17]:
import snowflake.connector
snowflake_username = 'DMOGARALA'
snowflake_password = 'Keerthi@1155182089'
snowflake_account = 'HFKYIDV-DS90906'
snowflake_warehouse = 'COMPUTE_WH'
snowflake_database = 'COVID19_EPIDEMIOLOGICAL_DATA'
snowflake_schema = 'PUBLIC'
snowflake_query = 'SELECT * FROM DEMOGRAPHICS;'

snowflake_details = {'user':snowflake_username,
    'password':snowflake_password,
    'account':snowflake_account,
    'database':snowflake_database,
    'schema':snowflake_schema,
    'warehouse':snowflake_warehouse,
    }


In [19]:
with snowflake.connector.connect(
        user=snowflake_details['user'],
        password=snowflake_details['password'],
        account=snowflake_details['account'],
        warehouse=snowflake_details['warehouse'],
        database=snowflake_details['database'],
        schema=snowflake_details['schema']
) as con:

  # Create a cursor object
  cursor = con.cursor()

  # Execute the query
  cursor.execute(snowflake_query)

  # Fetch all the results into a Pandas DataFrame
  data = cursor.fetch_pandas_all()

In [20]:
data

,ISO3166_1,ISO3166_2,FIPS,LATITUDE,LONGITUDE,STATE,COUNTY,TOTAL_POPULATION,TOTAL_MALE_POPULATION,TOTAL_FEMALE_POPULATION
0,US,NY,36049,43.782635,-75.218940,NY,Lewis County,27107,13671,13436
1,US,NY,36051,42.624741,-77.534340,NY,Livingston County,64622,32439,32183
2,US,NY,36053,42.776950,-75.343208,NY,Madison County,72089,35429,36660
3,US,NY,36055,43.075603,-77.385930,NY,Monroe County,749236,361925,387311
4,US,NY,36057,42.934388,-74.124411,NY,Montgomery County,49667,24468,25199
...,...,...,...,...,...,...,...,...,...,...
3135,US,TX,48193,31.657384,-97.986807,TX,Hamilton County,8232,4015,4217
3136,US,TX,48195,36.189358,-101.189414,TX,Hansford County,5552,2757,2795
3137,US,TX,48197,34.232102,-99.497176,TX,Hardeman County,3952,1934,2018
3138,US,TX,48199,30.311172,-94.134897,TX,Hardin County,55624,27484,28140


In [8]:
# Import necessary libraries
import pandas as pd
import snowflake.connector

# Function to connect to Snowflake and retrieve data
def get_snowflake_data(username, password, account, warehouse, database, schema, query):
    # Set up Snowflake connection parameters
    connection_params = {
        'user': username,
        'password': password,
        'account': account,
        'warehouse': warehouse,
        'database': database,
        'schema': schema
    }

    # Establish a connection to Snowflake
    with snowflake.connector.connect(**connection_params) as con:
        # Create a cursor object
        cursor = con.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch all the results into a Pandas DataFrame
        data = cursor.fetch_pandas_all()

    return data

# Function to summarize and export data to Excel
def exporting(data, excel_file):

    # Export the summary to an Excel file
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        data.to_excel(writer, sheet_name='Data', index=False)

# Configure your Snowflake account details
snowflake_username = 'DMOGARALA'
snowflake_password = 'Keerthi@1155182089'
snowflake_account = 'HFKYIDV-DS90906'
snowflake_warehouse = 'COMPUTE_WH'
snowflake_database = 'COVID19_EPIDEMIOLOGICAL_DATA'
snowflake_schema = 'PUBLIC'
snowflake_query = 'SELECT * FROM DEMOGRAPHICS;'

# Set the output Excel file name
output_excel_file = 'output_data_summary.xlsx'

# Get data from Snowflake
snowflake_data = get_snowflake_data(
    snowflake_username, snowflake_password, snowflake_account,
    snowflake_warehouse, snowflake_database, snowflake_schema, snowflake_query
)

# Summarize the data and export to Excel
summarize_and_export(snowflake_data, output_excel_file)

# Print a success message
print(f"Data has been successfully pulled from Snowflake, summarized, and exported to {output_excel_file}.")


Data has been successfully pulled from Snowflake, summarized, and exported to output_data_summary.xlsx.


### 2. Query some data

Assuming the source database has some data in it or that you can load some data into it from any source, query it.  Then, read that data into a Pandas dataframe.

In [6]:
df = con.cursor().execute('''
  select current_account(),
         current_timestamp(),
         current_database(),
         current_schema(),
         current_warehouse();''').fetch_pandas_all()

In [7]:
df

,CURRENT_ACCOUNT(),CURRENT_TIMESTAMP(),CURRENT_DATABASE(),CURRENT_SCHEMA(),CURRENT_WAREHOUSE()
0,UW52411,2023-12-11 13:31:57.904000-08:00,COVID19_EPIDEMIOLOGICAL_DATA,PUBLIC,COMPUTE_WH


### 3. Aggregate your data frame

Do some kind of aggregation on your data frame.  Something that makes sense and has some groups to it.  Don't just sum up one column for the entire data frame.  Be more creative than that.

### 4. Write to Excel

Use Pandas functions to write your summarized data out to a local Excel file.

---

## Submitting Your Work

Submit your work as usual